## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from subprocess import Popen, PIPE
import os
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode

client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 2
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [client.add_str(_nickname + 'avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_477_000]

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(20)
COEF_DOWN = str(5)
TARGET_CLAIM = str(2)

NODE_URL = 'https://rpc.space-pussy-1.cybernode.ai:443'
NETWORK = 'space-pussy-1'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(18)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1hxwv9eu9duxkuvluunxlayy9dhzwtm3e84fx47suf5jju04gumsqx7afgz'

INIT_GIFT_CONTRACT = False
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1fs5xwhralvmm49jdpy7grazcw8fsftu6d0pwn79xerkak65f83mqkn75jf'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

def execute_bash(bash_command: str, display_data: bool = False) -> [str, str]:
    _output, _error = Popen(bash_command, shell=True, stdout=PIPE, stderr=PIPE).communicate()
    if _error:
        display_data = True
    if display_data:
        print(bash_command)
        print(_output.decode('utf-8'))
        print(_error.decode('utf-8'))
    return _output.decode('utf-8'), _error.decode('utf-8')

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for _ in range(NUMBER_OF_PARTICIPANTS):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[0],
                  'nickname': NICKNAME_LIST[0],
                  'avatar': AVATAR_CID_LIST[0]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    # message_json = {
    #     "nickname": claim_item['nickname'],
    #     "gift_claiming_address": claim_item['ethereum_address'],
    #     "gift_amount": claim_item['amount'],
    #     "target_address": claim_item['bostrom_address']
    # }
    # claim_item['message'] = json.dumps(message_json)
    claim_item['message'] = 'QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV'

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1477000,john16443172440,QmQHSQFjJgssCY169nDzN4Dfi7Gf6eeSFExtMAcGqA8nBc,0x08d8dd519689b85d36fd318fa88bd5d1586c4d6b,0x0954aa3aac2ee8e3aeaed2343bf5bf407107780832d2...,bostrom1klp3t8kk7w924tw8et6gj7qtgfap8du0dqa0xe,cosmos1klp3t8kk7w924tw8et6gj7qtgfap8du0wnfuc7,guitar travel bench congress dumb ride casino ...,QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV,0x4c94599713ce86629ef3975d460429ac01ac0942bc57...,{'pub_key': 'Awr+wK9wc00Ur0pATcxG3XKeqm74MoXdL...,eyJwdWJfa2V5IjoiQXdyK3dLOXdjMDBVcjBwQVRjeEczWE...
1,1477000,john16443172440,QmQHSQFjJgssCY169nDzN4Dfi7Gf6eeSFExtMAcGqA8nBc,0x58d86dd6b9a3915eb91706f840fa3ff179cd8fa8,0xc1cb7f73a3d1e049b827c11d0fb16e4f30d27f6f79c6...,bostrom1tzj65n3emke5yx2ff93jhau8aennnnpjwpup8r,cosmos1tzj65n3emke5yx2ff93jhau8aennnnpjdjgjey,strike silent agent worry dolphin fantasy stoc...,QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV,0xe06da8bf81781d90ff52a2d9acf2522376803f238c8e...,{'pub_key': 'AudAUcXCTDqtGHM97PskQ0jmL1A5wkMNl...,eyJwdWJfa2V5IjoiQXVkQVVjWENURHF0R0hNOTdQc2tRMG...


### Instantiate Gift Contract

In [3]:
if INIT_GIFT_CONTRACT:
    init_output, init_error = execute_bash(
        f'''INIT='{{"owner":"{os.getenv('WALLET_ADDRESS')}", "passport":"{PASSPORT_CONTRACT_ADDRESS}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}' \
               && cyber tx wasm instantiate {GIFT_CODE_ID} "$INIT" --from $WALLET --amount {INITIAL_BALANCE}boot --label "cybergift test" -y --gas 3500000 --broadcast-mode block -o json --chain-id={NETWORK} --node={NODE_URL}''')
    if init_error:
        print(init_error)
    init_json = json.loads(init_output)
    gift_contract_address = \
        [event['attributes'][0]['value'] for event in init_json['logs'][0]['events'] if event['type'] == 'instantiate'][0]
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1fs5xwhralvmm49jdpy7grazcw8fsftu6d0pwn79xerkak65f83mqkn75jf


In [4]:
if INIT_PASSPORT_CONTRACT:
    init_output, init_error = execute_bash(
        f'''INIT='{{"name":"CPT", "minter":"{os.getenv('WALLET_ADDRESS')}", "owner":"{os.getenv('WALLET_ADDRESS')}", "symbol":"CPT"}}' \
               && cyber tx wasm instantiate {PASSPORT_CODE_ID} "$INIT" --from $WALLET --amount {INITIAL_BALANCE}boot --label "cybergift test" -y --gas 3500000 --broadcast-mode block -o json --chain-id={NETWORK} --node={NODE_URL}''')
    if init_error:
        print(init_error)
    init_json = json.loads(init_output)
    passport_contract_address = \
        [event['attributes'][0]['value'] for event in init_json['logs'][0]['events'] if event['type'] == 'instantiate'][0]
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1hxwv9eu9duxkuvluunxlayy9dhzwtm3e84fx47suf5jju04gumsqx7afgz


### Create Merkle Root

In [5]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [6]:
execute_bash(f'rm {ROOT_FILE_NAME}')
root_output, root_error = execute_bash(
    f'merkle-airdrop-cli generateRoot --file {ROOT_SOURCE_FILE_NAME} >> {ROOT_FILE_NAME} && cat {ROOT_FILE_NAME}')
root = root_output.replace('\n', '')
print(f'Merkle root: {root}')

Merkle root: 8eb0a8b9253b71e260553b20fef948bd9746cdfca4de6ec7a55d297d2d8bf185


In [7]:
root_register_output, root_register_error = \
    execute_bash(
        f'''REGISTER='{{"register_merkle_root":{{"merkle_root":"{root}"}}}}' \
            && CONTRACT="{gift_contract_address}" \
            && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -y --chain-id={NETWORK} --node={NODE_URL}''')

#### Get Merkle Root form the Gift Contract

In [8]:
print(
    execute_bash(
        f'''QUERY='{{"merkle_root": {{}}}}' \
            && cyber query wasm contract-state smart {gift_contract_address} "$QUERY" --chain-id={NETWORK} --node={NODE_URL}''')[0]
)

data:
  merkle_root: 8eb0a8b9253b71e260553b20fef948bd9746cdfca4de6ec7a55d297d2d8bf185



### Create Passports

In [9]:
def create_passport(claim_row: pd.Series, display_data: bool = False):
    return execute_bash(
                f'''REGISTER='{{"create_passport":{{"avatar":"{claim_row["avatar"]}", "nickname":"{claim_row["nickname"]}"}}}}' \
                    && CONTRACT="{passport_contract_address}" \
                    && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -y --gas=400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        create_passport(row, display_data=True)
    else:
        create_passport(row)

0it [00:00, ?it/s]

REGISTER='{"create_passport":{"avatar":"QmQHSQFjJgssCY169nDzN4Dfi7Gf6eeSFExtMAcGqA8nBc", "nickname":"john16443172440"}}'                     && CONTRACT="bostrom1hxwv9eu9duxkuvluunxlayy9dhzwtm3e84fx47suf5jju04gumsqx7afgz"                     && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -y --gas=400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
code: 0
codespace: ""
data: 0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374
gas_used: "227940"
gas_wanted: "400000"
height: "1175691"
info: ""
logs:
- events:
  - attributes:
    - key: particleFrom
      value: QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf
    - key: particleTo
      value: QmTc6XV8KUrvsY1j2TJgXRe6FmZsBtKSr9USKN1vXN9vMZ
    - key: particleFrom
      value: QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf
    - key: particleTo
      value: QmQHSQFjJgssCY169nDzN4Dfi7Gf6eeSFExtMAcGqA8nBc
    - key: particleFrom
      value: QmTc6

### Proof (Add) Address to Passports

In [10]:
def proof_address(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
            f'''REGISTER='{{"proof_address":{{"nickname":"{claim_row["nickname"]}", "address":"{claim_row[network + "_address"]}", "signature":"{claim_row[network + "_message_signature"]}"}}}}' \
                && CONTRACT="{passport_contract_address}" \
                && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -y --gas=400000 --chain-id={NETWORK} --node={NODE_URL}''',
            display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 1:
        # proof_address(row, display_data=True)
        proof_address(row, network='cosmos', display_data=True)
    else:
        # proof_address(row)
        # proof_address(row, network='cosmos', display_data=True)
        pass

0it [00:00, ?it/s]

REGISTER='{"proof_address":{"nickname":"john16443172440", "address":"cosmos1tzj65n3emke5yx2ff93jhau8aennnnpjdjgjey", "signature":"eyJwdWJfa2V5IjoiQXVkQVVjWENURHF0R0hNOTdQc2tRMGptTDFBNXdrTU5sSFp1QXRwNTZyOUEiLCJzaWduYXR1cmUiOiJtQmw2Y1FUSTMzRHF1S211NWFBNHYxblcyQUxLKzdyWWtaM0lXNFVvK3Znc3NYOHJjN0hoV05od0ZnMmZQTXlZbnhMSjJzZVdQczVnbnh6RFNlNlEvQT09In0="}}'                 && CONTRACT="bostrom1hxwv9eu9duxkuvluunxlayy9dhzwtm3e84fx47suf5jju04gumsqx7afgz"                 && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -y --gas=400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
code: 5
codespace: wasm
data: ""
gas_used: "120020"
gas_wanted: "400000"
height: "1175693"
info: ""
logs: []
raw_log: 'failed to execute message; message index: 0: Address is not eligible to
  claim airdrop, Generic error: addr_validate errored: invalid Bech32 prefix; expected
  bostrom, got cosmos: execute wasm contract failed'
timestamp: ""
tx: null
txhash

### Generate Proofs and Claim

In [11]:
def generate_proof_and_claim(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    proof_output, _ = execute_bash(f'merkle-airdrop-cli generateProofs --file {ROOT_SOURCE_FILE_NAME} --address {claim_row[network + "_address"]} --amount {claim_row.amount}',
                                   display_data=display_data)
    proof = proof_output.replace('\n','').replace(" ",'').replace("'", '"')
    return execute_bash(
                f'''CLAIM='{{"claim":{{"nickname":"{claim_row['nickname']}", "gift_claiming_address":"{claim_row[network + "_address"]}", "gift_amount":"{claim_row.amount}", "proof":{proof}}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        generate_proof_and_claim(row, network='cosmos', display_data=True)
    else:
        # generate_proof_and_claim(row)
        pass


0it [00:00, ?it/s]

merkle-airdrop-cli generateProofs --file root_testing_source_2_addresses.json --address cosmos1klp3t8kk7w924tw8et6gj7qtgfap8du0wnfuc7 --amount 1477000
[
  '98bf79467efe4b717779d06490289e9a793e51221cef4b142cb5adca5ee6381c',
  '4cdd4653d3056ea3d1096c592ae3651deff280154acc987aa535d752dd3f5ffb'
]


CLAIM='{"claim":{"nickname":"john16443172440", "gift_claiming_address":"cosmos1klp3t8kk7w924tw8et6gj7qtgfap8du0wnfuc7", "gift_amount":"1477000", "proof":["98bf79467efe4b717779d06490289e9a793e51221cef4b142cb5adca5ee6381c","4cdd4653d3056ea3d1096c592ae3651deff280154acc987aa535d752dd3f5ffb"]}}'                     && cyber tx wasm execute bostrom1fs5xwhralvmm49jdpy7grazcw8fsftu6d0pwn79xerkak65f83mqkn75jf "$CLAIM" --from $WALLET --broadcast-mode block -y --gas 400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
code: 5
codespace: wasm
data: ""
gas_used: "184240"
gas_wanted: "400000"
height: "1175694"
info: ""
logs: []
raw_log: 'failed to execute message; message index: 0